# Importation des bibliothéques

In [26]:
import pandas as pd
import glob

# Lecture des fichiers du dossier graph G99

In [27]:
!ls /data/bd/dataset/proteine/80_80/G99

components.parquet		    graph_partition_by_query_id
graph				    graph_partitionned
graph_deduplicated2		    graph_partitionned_components
graph_nodeID_only		    graph_removed
graph_nodeID_only_component	    graph_with_components
graph_nodeID_only_partitionned	    node_with_comp_and_annotation
graph_nodeID_only_partitionned_200


In [28]:
parquet_folder = "/data/bd/dataset/proteine/80_80/G99/graph/"

# Lire tous les fichiers Parquet dans une liste de DataFrames
dfs = [pd.read_parquet(fichier) for fichier in glob.glob(parquet_folder + '*.parquet')]

# Concaténer les DataFrames en un seul DataFrame
df = pd.concat(dfs, ignore_index=True)

# Afficher les premières lignes du DataFrame
df.head()

,query_id,query_length,target_id,target_length,match_length,percent_identity,e_value,relative_sim
0,17253890:3,205,132997206:3,281,231,87.9,1.300000e-105,0.99
1,17253916:4,97,15406465:2,321,96,100.0,6.100000e-47,0.99
2,17253916:4,97,METdb_00033-1-DN42202-c1-g7-i1.p1,392,96,100.0,8.200000e-47,0.99
3,17253916:4,97,METdb_00033-1-DN42202-c1-g7-i2.p1,392,96,100.0,8.200000e-47,0.99
4,17253916:4,97,METdb_00369-1-DN52456-c0-g1-i1.p1,392,96,100.0,8.200000e-47,0.99


In [29]:
df = df[['target_id', 'query_id']]
df.shape

(359798439, 2)

In [31]:
d = 200
new_df = df
# Ajout d'une nouvelle colonne 'partition' basée sur le modulo de target_id
new_df['partition'] = new_df['target_id'].apply(lambda x: hash(x) % d)
partition_freq = new_df.groupby('partition').size().reset_index(name='count')
partition_freq.head()

,partition,count
0,0,1784868
1,1,1787699
2,2,1710721
3,3,1758926
4,4,1799652


In [32]:
partitions = partition_freq['partition'].unique()
for partition in partitions:
    df_partition = new_df[new_df['partition'] == partition]
    file_path = f"/data/bd/dataset/proteine/80_80/G99/graph_partitionned/200_partitions/partition_{partition}.parquet"
    df_partition.to_parquet(file_path, index=False, engine='pyarrow')

# On varie le degree de parallélisme (num_partition)

In [25]:
degree_parallelisme = [10,20, 30, 40, 50, 60]

for d in degree_parallelisme :
    print("degree de parallélisme : ",d)
    new_df = df
    # Ajout d'une nouvelle colonne 'partition' basée sur le modulo de target_id
    new_df['partition'] = new_df['target_id'].apply(lambda x: hash(x) % d)
    partition_freq = new_df.groupby('partition').size().reset_index(name='count')
    
    partitions = partition_freq['partition'].unique()
    for partition in partitions:
        df_partition = new_df[new_df['partition'] == partition]
        file_path = f"/data/bd/dataset/proteine/80_80/G99/graph_partitionned/{d}_partitions/partition_{partition}.parquet"
        df_partition.to_parquet(file_path, index=False, engine='pyarrow')
    

degree de parallélisme :  10
degree de parallélisme :  20
degree de parallélisme :  30
degree de parallélisme :  40
degree de parallélisme :  50
degree de parallélisme :  60
